<a href="https://colab.research.google.com/github/roshanappa/Time-Series-Analysis-/blob/master/Data_Transformation_Techniques_In_The_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [106]:
from zipfile import ZipFile

In [107]:
with ZipFile('/content/64789_128364_bundle_archive.zip',mode='r') as zip:
  zip.extractall()
  print('Done')

Done


In [108]:
data_birth = pd.read_csv('/content/daily-total-female-births-CA.csv',parse_dates=[0],squeeze=True)

In [109]:
data_birth.head()

,date,births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


In [110]:
data_birth.tail()

,date,births
360,1959-12-27,37
361,1959-12-28,52
362,1959-12-29,48
363,1959-12-30,55
364,1959-12-31,50


In [111]:
data_birth.isnull().sum()

date      0
births    0
dtype: int64

In [112]:
data_birth.shape

(365, 2)

In [113]:
data_birth.dtypes

date      datetime64[ns]
births             int64
dtype: object

In [114]:
data_birth.set_index(data_birth['date'],inplace=True)

In [115]:
data_birth.drop(columns=['date'],inplace=True)

In [116]:
data_birth.head()

,births
date,
1959-01-01,35
1959-01-02,32
1959-01-03,30
1959-01-04,31
1959-01-05,44


# Normalization.

Whenver data having the abnormal type of the scaling in that model will not gives the similar imporatant while training data.

**Normalization = x-min(x)/max(x)-min(x)**

During the that period we have to maintain the data in the common scale so that model will gives us better and reliable output.

Standardized the scale and normalized the data that meeans mean equal to zero and standard deviation equal to one.

In [117]:
data_birth.min()

births    23
dtype: int64

In [118]:
data_birth.max()

births    73
dtype: int64

In [119]:
#so in the case of 37
(37-23)/(73-23)

0.28

In [120]:
from sklearn.preprocessing import MinMaxScaler

In [121]:
scaler = MinMaxScaler(feature_range=(0,1))
scalled_data = scaler.fit(data_birth)

In [122]:
print(scalled_data.data_max_)
print(scalled_data.data_min_)
print(scalled_data.data_range_)

[73.]
[23.]
[50.]


In [123]:
birth_normalized = scaler.transform(data_birth)

In [124]:
birth_normalized[0]

array([0.24])

In [125]:
birth_normalized.shape

(365, 1)

In [126]:
data_train = birth_normalized[:330]
data_test  = birth_normalized[330:]

In [127]:
from statsmodels.tsa.arima_model import ARIMA

In [128]:
model = ARIMA(data_train,(2,1,3))
fit_model =model.fit()

In [129]:
fit_model.aic

-343.1628893804674

In [130]:
predictions = fit_model.forecast(steps=len(data_test))[0]
predictions

array([0.44607755, 0.46622011, 0.41298646, 0.4318446 , 0.46994912,
       0.42687009, 0.42179587, 0.46688497, 0.44151052, 0.41761997,
       0.45904807, 0.45392106, 0.41947947, 0.44913816, 0.46208218,
       0.42622862, 0.4398488 , 0.46521045, 0.43587295, 0.43329651,
       0.46372052, 0.44613299, 0.4306668 , 0.458939  , 0.45497539,
       0.43211634, 0.45267206, 0.46100393, 0.43690983, 0.44674267,
       0.46365065, 0.44372353, 0.44260074, 0.46315948, 0.45102362])

In [131]:
data_test[:5]

array([[0.56],
       [0.46],
       [0.42],
       [0.58],
       [0.44]])

In [132]:
from sklearn.metrics import mean_squared_error

In [133]:
np.sqrt(mean_squared_error(data_test,predictions))

0.1372006151832518

# Reverse Transformation For The Forecast

In [134]:
birth_reshape = predictions.reshape(len(data_test),1)

In [135]:
birth_forecast_reverse = scaler.inverse_transform(birth_reshape)

In [139]:
birth_forecast_reverse[:7]

array([[45.30387753],
       [46.31100566],
       [43.64932316],
       [44.59222995],
       [46.49745598],
       [44.34350428],
       [44.08979333]])

# Reverse Transformation for the test_data.

In [144]:
test_data_reshape = data_test.reshape(len(data_test),1)

In [145]:
test_data_reverse = scaler.inverse_transform(test_data_reshape)

In [147]:
test_data_reverse[:5]

array([[51.],
       [46.],
       [44.],
       [52.],
       [45.]])

In [148]:
np.sqrt(mean_squared_error(test_data_reverse,birth_forecast_reverse))

6.860030759162588

# Simlilar Step for the StandardScaler also.